In [1]:
#Get the first page to extract page numbers
import requests, re
from bs4 import BeautifulSoup

r=requests.get("http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=")
c=r.content

soup=BeautifulSoup(c,"html.parser")

all=soup.find_all("div",{"class":"cassetteitem"})

all[0].find("span",{"class":"cassetteitem_other-emphasis ui-text--bold"}).text.replace("\n","").replace(" ","")
test = all[0].find("li",{"class","cassetteitem_detail-col1"})
print(test)

page_nr=soup.find_all("ol",{"class":"pagination-parts"})[-1].text
page_nr = [int(s) for s in page_nr.split() if s.isdigit()]
page_nr = page_nr[len(page_nr)-1]

print(page_nr,"pages were found")


<li class="cassetteitem_detail-col1">東京都文京区白山１</li>
533 pages were found


In [2]:
import re

l=[]
base_url='http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2='

items = []
n = 0

for page in range(1,5,1):
    #print(    )
    r=requests.get(base_url+'&pn='+str(page))
    c=r.content
    #c=r.json()["list"]
    soup=BeautifulSoup(c,"html.parser")
    all = soup.find_all(lambda tag: tag.name == 'div' and 
                                   tag.get('class') == ['cassetteitem'])
    #all=soup.find_all("div",{"class":"cassetteitem"})
    

    
    for item in all:
        d={}
        d["-Title"]=item.find("div",{"class","cassetteitem_content-title"}).text
        d["Locality"]=item.find("li",{"class","cassetteitem_detail-col1"}).text
        # print(item.find_all("li",{"class","cassetteitem_detail-col1"})[0].text
        d["Price"]=item.find("span",{"class","cassetteitem_other-emphasis ui-text--bold"}).text.replace("\n","").replace(" ","")

        #Rooms
        rooms = item.find("table",{"class","cassetteitem_other"}).text
        if 'ワンルーム' in rooms:
            d["Rooms"]=1
        elif '1K' in rooms:
            d["Rooms"]=1
        elif '2K' in rooms:
            d["Rooms"]=2
        else:
            d["Rooms"]="Unknown"
        
        table = item.find_all("table",{"class","cassetteitem_other"})[0]
        nums = re.findall(r'\d+(?:\.\d+)?', str(table))
        d["Size"]=nums[-6]+" m2"
        
        d["Link"]=item.find("td",{"class","ui-text--midium ui-text--bold"}).text
        
        #d[]=item.find_all("",{"",""})[0].text

        l.append(d)

In [3]:
l

[{'-Title': 'デュプレクス白山',
  'Link': '\n詳細を見る',
  'Locality': '東京都文京区白山１',
  'Price': '1.4万円',
  'Rooms': 'Unknown',
  'Size': '52.08 m2'},
 {'-Title': '司荘',
  'Link': '\n詳細を見る',
  'Locality': '東京都新宿区北新宿２',
  'Price': '2万円',
  'Rooms': 1,
  'Size': '9 m2'},
 {'-Title': '東京メトロ丸ノ内線 中野坂上駅 2階建 築53年',
  'Link': '\n詳細を見る',
  'Locality': '東京都新宿区北新宿２',
  'Price': '2万円',
  'Rooms': 1,
  'Size': '6.2 m2'},
 {'-Title': '京王新線 幡ヶ谷駅 2階建 築50年',
  'Link': '\n詳細を見る',
  'Locality': '東京都渋谷区幡ヶ谷３',
  'Price': '2.4万円',
  'Rooms': 1,
  'Size': '9.31 m2'},
 {'-Title': '光風荘',
  'Link': '\n詳細を見る',
  'Locality': '東京都文京区小日向３',
  'Price': '2.5万円',
  'Rooms': 1,
  'Size': '12 m2'},
 {'-Title': 'ＪＲ中央線 東中野駅 2階建 築48年',
  'Link': '\n詳細を見る',
  'Locality': '東京都新宿区北新宿４',
  'Price': '2.4万円',
  'Rooms': 1,
  'Size': '10.74 m2'},
 {'-Title': '樟川荘',
  'Link': '\n詳細を見る',
  'Locality': '東京都新宿区北新宿３',
  'Price': '2.8万円',
  'Rooms': 1,
  'Size': '10 m2'},
 {'-Title': '東京メトロ千代田線 千駄木駅 2階建 築43年',
  'Link': '\n詳細を見る',
  'Locality': '東京都文

In [5]:
import pandas
df=pandas.DataFrame(l)

In [6]:
df

,-Title,Link,Locality,Price,Rooms,Size
0,デュプレクス白山,\n詳細を見る,東京都文京区白山１,1.4万円,Unknown,52.08 m2
1,司荘,\n詳細を見る,東京都新宿区北新宿２,2万円,1,9 m2
2,東京メトロ丸ノ内線 中野坂上駅 2階建 築53年,\n詳細を見る,東京都新宿区北新宿２,2万円,1,6.2 m2
3,京王新線 幡ヶ谷駅 2階建 築50年,\n詳細を見る,東京都渋谷区幡ヶ谷３,2.4万円,1,9.31 m2
4,光風荘,\n詳細を見る,東京都文京区小日向３,2.5万円,1,12 m2
5,ＪＲ中央線 東中野駅 2階建 築48年,\n詳細を見る,東京都新宿区北新宿４,2.4万円,1,10.74 m2
6,樟川荘,\n詳細を見る,東京都新宿区北新宿３,2.8万円,1,10 m2
7,東京メトロ千代田線 千駄木駅 2階建 築43年,\n詳細を見る,東京都文京区千駄木１,2.6万円,1,10 m2
8,都営大江戸線 落合南長崎駅 2階建 築41年,\n詳細を見る,東京都新宿区西落合３,2.6万円,1,9.72 m2
9,大和荘,\n詳細を見る,東京都文京区本駒込４,2.5万円,1,10.5 m2


In [107]:
df.to_csv("Output.csv")